In [3]:
import numpy as np
import os
from pathlib import Path

import pandas as pd
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import scipy.io as scio
import importlib
from scipy import sparse

load the dl_model

In [4]:
species = "Saccharomyces_cerevisiae"
generation = 150
enzymedataFile = "data/Saccharomyces_cerevisiae_dl.mat"
#enzymedataFile = "data/model.mat"
z = scio.loadmat(enzymedataFile)
enzymedata = z['enzymedata'][0,0]
max_growth = z['max_growth']
growthdata = z['growthdata']
model = z['model'][0,0]
strain = z['strain']
rxn2block = z['rxn2block']

#split the training data
growthdata_train = growthdata[1:17:2,:]
growthdata_test = growthdata[0:17:2,:]
max_growth_train = max_growth[1:8:2,:]
max_growth_test = max_growth[0:8:2,:]

In [9]:
import scipy.io as io
F = sumBiomass(model)
#calculate the prot_weight
F = 0.68651398  #怎么从sumBiomass中导出结果，matlab版本为0.46
tot_prot_weight = F*0.5
if strain == 'Kluyveromyces_marxianus':
    tot_prot_weight = 0.325
elif strain == 'Kluyveromyces_lactis':
    tot_prot_weight = 0.245

P -> [[0.68651398]] g/gDW
X ->[[0.68651398]] gDW/DW
Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-21


No defined compartments in model model__. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, ce, e, er, erm, g, gm, lp, m, mm, n, p, v, vm


Growth = 0.08158852335573963 1/h


In [10]:
proc = 18
numPerGeneration = 126 #126/18 = 7
rejectnum = 0.2
generation = 100

In [11]:
proc = 1
sample_generation = 1
j = 1
kcat_random_all = enzymedata['kcat']
nstep = sample_generation/proc
nstep = int(nstep)
rmse_final = np.zeros((1, nstep))
kcat_sample = kcat_random_all[:, ((j - 1) * nstep ):j * nstep] #==[:,1:1],但只有一列

#get carbonnum for each exchange rxn to further calculation of error
#一般都有（未确定是否有问题）
#if not len(model["excarbon"]):  ##检查model中是否有excarbon, ~is_field
#   model = addCarbonNum(model)  ##补写function

for k in range(nstep):
    print('nstep:' + str(k+1) + '/' + str(nstep))
    kcat_random = kcat_sample[:, k]
    prot_cost_info_value = [enzymedata["MW"][i]/kcat_random[i] for i in range(len(kcat_random))]
    prot_cost_info_id = enzymedata["rxn_list"]

nstep:1/1


In [12]:
objective = 'r_2111'
osenseStr = 'max'

In [ ]:
%%time
exp_d = []
sim = []
constrain = True
simulated = np.zeros((len(growthdata[:, 0]), 9))
for i in range(len(growthdata[:, 0])):
    exp = np.array(growthdata[:, 2:11])
    exp = exp * [1, -1, 1, 1, 1, 1, 1, 1, -1]
    ex_mets = ['growth', str(growthdata[i, 1])+' exchange', 'acetate exchange', 'ethanol exchange',
               'glycerol exchange','pyruvate exchange', 'ethyl acetate exchange', 'carbon dioxide exchange',
               'oxygen exchange']
    rxnNames = model['rxnNames']
    #idx = [i for i, ex_mets in enumerate(rxnNames) if ex_mets != 0]
    #idx = rxnNames.find(ex_mets)  ##[Lia,Locb] = ismember(A,B)
    idx = []
    temp = []
    m = 0
    for k in range(len(rxnNames)):
        if rxnNames[k] in ex_mets:
            temp.append(k)
    idx.append(temp)
    model_tmp = model
    model_tmp = changeMedia(model_tmp, 'D-glucose', growthdata[i, 15])
    for k in range(len(model_tmp['rxns'])):
        if 'r_1634' == model_tmp['rxns'][k]:
            model_tmp['lb'][k] = 0
            model_tmp['ub'][k] = 0
    for k in range(len(model_tmp['rxns'])):
        if 'r_1631' == model_tmp['rxns'][k]:
            model_tmp['lb'][k] = 0
            model_tmp['ub'][k] = 0

    if growthdata[i, 13] == "anaerobic" or growthdata[i, 13] == "limited":
        model_tmp = anaerobicModel(model_tmp)
    if growthdata[i, 13] == "limited":
        for k in range(len(model_tmp["rxnNames"])):
            if model_tmp["rxnNames"][k] == 'oxygen exchange':  # if model_tmp["rxnNames"] == 'oxygen exchange'
                model_tmp["lb"][k] = -5

    for x in range(len(model_tmp["rxns"])):
        if not constrain:
            if model_tmp["rxns"][x] == 'r_1714':
                model_tmp["lb"][x] = 0
            elif model_tmp["rxns"][x] == model_tmp["rxns"][idx[1]]:
                model_tmp["lb"][x] = -1000
        else:
            if model_tmp["rxns"][x] == 'r_1714':
                model_tmp["lb"][x] = 0
                model_tmp["lb"][np.array(idx)[:,1]] = exp[i, 1]
    tmp = []
    for t in range(len(exp[i,:].astype('float'))):
        if not np.isnan(any(exp[i,t].astype('float'))) and not np.isinf(any(exp[i,t].astype('float'))):
            tmp.append(t)



C:\Users\sherr\AppData\Local\Temp\ipykernel_18868\2078549751.py:102: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if s in model['rxns'][idx]:


In [6]:
def getFraction(model, data, compType, X):
    # define pseudoreaction name
    rxnName = compType+' pseudoreaction'
    rxnName = rxnName.replace('P', 'protein')
    rxnName = rxnName.replace('C', 'carbohydrate')
    rxnName = rxnName.replace('N', 'biomass')
    rxnName = rxnName.replace('L', 'lipid backbone')
    rxnName = rxnName.replace('R', 'RNA')
    rxnName = rxnName.replace('D', 'DNA')
    rxnName = rxnName.replace('I', 'ion')
    rxnName = rxnName.replace('F', 'cofactor')

    # add up fraction
    rxnPos = []
    for i in range(len(model['rxnNames'])):
        if rxnName == model['rxnNames'][i]:
            temp = 1
        else:
            temp = 0
        rxnPos.append(temp)
    index = [i for i, e in enumerate(rxnPos) if e!=0]
    if np.nonzero(rxnPos) != 0:
        sub = model['S']
        isSub = sub.getcol(index[0])<0 #substrates in pseudo-rxn
        if compType == 'L':
            F = - sum(sub[isSub, rxnPoS])  #g/gDW   算出来是空？应为4.2336
        else:
            F = 0
            # add up all components:
            for i in range(len(model['mets'])):
                mets = model['mets']
                pos = []
                if mets[i] in data['mets']:
                    temp = 1
                else:
                    temp = 0
                pos.append(temp)
                if isSub[i] and sum(pos) == 1:
                    if compType == 'I' or compType == 'F':
                        MW = data['MWs'][pos]
                    else:
                        MW = data['MWs'][pos] - 18
                    abundance = -sub[i,index]*MW/1000
                    F = F + abundance
        X = X + F

        print(str(compType) + ' -> ' + str(F) + " g/gDW")
    else:
        print(str(compType) + " do not exist")
        F = 0
        X = X + F
    print("X ->"+str(X)+" gDW/DW")
    return X, F

def sumBiomass(model):
    biomassCompDataFile = "data/biomassCompData.mat"
    biomassCompData = io.loadmat(biomassCompDataFile)
    biomassCompData = biomassCompData['data'][0, 0]

    getFraction(model,biomassCompData,'P',0)
    #getFraction(model,biomassCompData,'C',X)
    #getFraction(model,biomassCompData,'R',X)
    #getFraction(model,biomassCompData,'D',X)
    #getFraction(model,biomassCompData,'L',X)
    #getFraction(model,biomassCompData,'I',X)
    #getFraction(model,biomassCompData,'F',X)

    #print("X ->"+str(X)+" gDW/DW")
    io.savemat('model__.mat', {'model__': model})
    model__ = load_matlab_model("model__.mat")
    sol = model__.optimize()
    print("Growth = " + str(sol.objective_value) + " 1/h")

In [7]:
def changeMedia(model, c_source, media, anox = False, flux = -1000):
    c_source = c_source + ' exchange'
    # first block any uptake

    # model = setParam(model,'eq',model.rxns(rxnidx),0);
    #exchange = getExchangeRxns(model)
    #idx = model['rxnNames'].find('EX_protein_pool')
    #exchange = set(exchange).difference(set(idx))
    #model['lb'][exchange] = 0
    pos = []
    c_source = 'D-glucose exchange'
    for i in range(len(model['rxnNames'])):
        if model['rxnNames'][i] == c_source:
            pos.append(i)

    # The media will define which rxns to fix:
    if media == 'YEP':
        N = 25  # Aminoacids + Nucleotides
    elif media == 'MAA':
        N = 21  # Aminoacids
    elif media == 'MIN':
        N = 1  # Only the carbon source
    elif media == 'MIN+His':
        N = 1  # Only the carbon source
        model.reactions.get_by_id('r_1893').lower_bound = -0.08  # Histidine exchange
    elif media == 'MIN+Arg':
        N = 1  # Only the carbon source
        model.reactions.get_by_id('r_1879').lower_bound = -0.08  # L-arginine exchange
    elif media == 'MIN+Citrate':
        N = 1  # Only the carbon source
        model.reactions.get_by_id('r_1687').lower_bound = -0.08  # citrate exchange
    # LB parameter (manually optimized for glucose on Min+AA):
    b = -0.08
    # LB parameter (manually optimized for glucose complex media):
    c = -2
    flux = np.array(flux)
    # Define fluxes in case of ec model:
    if N > 1:
        flux = b * np.ones((1, N))
        if N > 21:
            flux[21:25] = c;
    flux = -1000 #flux[0] = -1000

    # Fix values as LBs:
    for i in range(N):
        model['lb'][pos[i]] = flux #flux[i]

    # Allow uptake of essential components
    for i in range(len(model['rxns'])):
        if model['rxns'][i] == 'r_1654':
            model['lb'][i] = -1000 # 'ammonium exchange';
        if model['rxns'][i] == 'r_2100':
            model['lb'][i] = -1000 # 'water exchange' ;
        if model['rxns'][i] == 'r_1861':
            model['lb'][i] = -1000 # 'iron(2+) exchange';
        if model['rxns'][i] == 'r_1992':
            model['lb'][i] = -1000 # 'oxygen exchange';
        if model['rxns'][i] == 'r_2005':
            model['lb'][i] = -1000 # 'phosphate exchange';
        if model['rxns'][i] == 'r_2060':
            model['lb'][i] = -1000 # 'sulphate exchange';
        if model['rxns'][i] == 'r_1832':
            model['lb'][i] = -1000 # 'H+ exchange' ;
        if model['rxns'][i] == 'r_4593':
            model['lb'][i] = -1000 # 'chloride exchange' ;
        if model['rxns'][i] == 'r_4595':
            model['lb'][i] = -1000 # Mn(2+) exchange
        if model['rxns'][i] == 'r_4596':
            model['lb'][i] = -1000 # Zn(2+ exchange
        if model['rxns'][i] == 'r_4597':
            model['lb'][i] = -1000 # Mg(2+) exchange
        if model['rxns'][i] == 'r_2049':
            model['lb'][i] = -1000 # sodium exchange
        if model['rxns'][i] == 'r_4594':
            model['lb'][i] = -1000 # Cu(2+) exchange
        if model['rxns'][i] == 'r_4600':
            model['lb'][i] = -1000 # Ca(2+) exchange
        if model['rxns'][i] == 'r_2020':
            model['lb'][i] = -1000 # potassium exchange

    # Block some production fluxes
        if model['rxns'][i] == 'r_1663':
            model['ub'][i] = 0 # bicarbonate exchange
        if model['rxns'][i] == 'r_4062':
            model['ub'][i] = 0 # lipid backbone exchange
        if model['rxns'][i] == 'r_4064':
            model['ub'][i] = 0 # lipid chain exchange

    # Allow biomass production
        if model['rxns'][i] == 'r_2111':
            model['ub'][i] = 1000 # growth

    if anox == 'anaerobic':
        1
        model = anerobicModel(model)

    return model


In [8]:
def anaerobicModel(model):
    # 1th change: Refit GAM and NGAM to exp. data, change biomass composition
    GAM = 58.1988
    strain = str(model['id']).replace(' specific model genereted from panYeast', '')
    if strain == 'Candida_glabrata' or strain == 'Candida_parapsilosis':
        GAM = 30

    NGAM = 0
    model = changeGAM(model, GAM, NGAM)
    # 2nd change: Removes the requirement of heme a in the biomass equation
    mets = ['s_3714[c]', 's_1198[c]', 's_1203[c]', 's_1207[c]', 's_1212[c]', 's_0529[c]']
    met_index = []
    for i in range(len(model['mets'])):
        if model['mets'][i] in mets:
            met_index.append(i)
    rxn_index = []
    for i in range(len(model['rxns'])):
        if model['rxns'][i] == 'r_4598':
            rxn_index.append(i)
    model['S'][met_index, rxn_index] = 0
    # 3st change: Changes media to anaerobic
    for s in range(len(model['rxns'])):
        if model['rxns'][s] == 'r_1992':
            model['lb'][s] = 0
        elif model['rxns'][s] == 'r_1757':
            model['lb'][s] = -1000
        elif model['rxns'][s] == 'r_1994':
            model['lb'][s] = -1000
        elif model['rxns'][s] == 'r_1915':
            model['lb'][s] = -1000
        elif model['rxns'][s] == 'r_2106':
            model['lb'][s] = -1000
        elif model['rxns'][s] == 'r_2134':
            model['lb'][s] = -1000
        elif model['rxns'][s] == 'r_2137':
            model['lb'][s] = -1000
        elif model['rxns'][s] == 'r_2189':
            model['lb'][s] = -1000

    # extra media set up for ura1 original but also the anaerobic growth media
    if len(model["id"]):
        strain = str(model['id']).replace(' specific model genereted from panYeast', '')
    species_onlyura9 = [
        'Alloascoidea_hylecoeti;Ambrosiozyma_kashinagacola;Ambrosiozyma_monospora;Arxula_adeninivorans;Ascoidea_asiatica;Ascoidea_rubescens;Ashbya_aceri;Aspergillus_nidulans;Babjeviella_inositovora;Brettanomyces_anomalus;Candida_albicans;Candida_apicola;Candida_arabinofermentans;Candida_auris;Candida_boidinii_JCM9604;Candida_carpophila;Candida_dubliniensis;Candida_glabrata;Candida_homilentoma;Candida_infanticola;Candida_intermedia;Candida_orthopsilosis;Candida_parapsilosis;Candida_sorboxylosa;Candida_succiphila;Candida_tanzawaensis;Candida_tenuis;Candida_tropicalis;Candida_versatilis;Clavispora_lusitaniae;Cyberlindnera_fabianii_JCM3601;Cyberlindnera_jadinii;Debaryomyces_hansenii;Dekkera_bruxellensis;Eremothecium_coryli;Eremothecium_cymbalariae;Eremothecium_gossypii;Eremothecium_sinecaudum;Geotrichum_candidum;Hanseniaspora_uvarum;Hanseniaspora_valbyensis;Hanseniaspora_vinae;Hyphopichia_burtonii;Komagataella_pastoris;Kuraishia_capsulata;Lipomyces_starkeyi;Lodderomyces_elongisporus;Metschnikowia_aberdeeniae;Metschnikowia_arizonensis;Metschnikowia_bicuspidata;Metschnikowia_borealis;Metschnikowia_bowlesiae;Metschnikowia_cerradonensis;Metschnikowia_continentalis;Metschnikowia_dekortum;Metschnikowia_drakensbergensis;Metschnikowia_hamakuensis;Metschnikowia_hawaiiensis;Metschnikowia_hibisci;Metschnikowia_ipomoeae;Metschnikowia_kamakouana;Metschnikowia_kipukae;Metschnikowia_lockheadii;Metschnikowia_matae;Metschnikowia_matae_maris;Metschnikowia_mauinuiana;Metschnikowia_proteae;Metschnikowia_santaceciliae;Metschnikowia_shivogae;Metschnikowia_similis;Meyerozyma_guilliermondii;Millerozyma_acaciae;Nadsonia_fulvescens_var_elongata;Nakaseomyces_bracarensis;Nakaseomyces_castellii;Nakaseomyces_delphensis;Nakaseomyces_nivariensis;Nakazawaea_peltata;Ogataea_methanolica;Ogataea_parapolymorpha;Ogataea_polymorpha;Pachysolen_tannophilus;Pichia_membranifaciens;Priceomyces_haplophilus;Saccharomycopsis_malanga;Saprochaete_clavata;Scheffersomyces_lignosus;Scheffersomyces_stipitis;Schizosaccharomyces_pombe;Spathaspora_arborariae;Spathaspora_girioi;Spathaspora_gorwiae;Spathaspora_hagerdaliae;Spathaspora_passalidarum;Sporopachydermia_quercuum;Starmerella_bombicola_JCM9596;Sugiyamaella_lignohabitans;Tortispora_caseinolytica;Vanderwaltozyma_polyspora;Wickerhamia_fluorescens;Wickerhamiella_domercqiae;Wickerhamomyces_anomalus;Wickerhamomyces_ciferrii;Yarrowia_deformans;Yarrowia_keelungensis;Yarrowia_lipolytica;yHMPu5000026124_Ogataea_henricii;yHMPu5000026137_Ambrosiozyma_ambrosiae;yHMPu5000026142_Citeromyces_matritensis;yHMPu5000026145_Ambrosiozyma_vanderkliftii;yHMPu5000026197_Brettanomyces_custersianus;yHMPu5000026274_Komagataella_populi;yHMPu5000034594_Starmera_quercuum;yHMPu5000034597_Candida_stellimalicola;yHMPu5000034604_Sporopachydermia_lactativora;yHMPu5000034605_Spencermartinsiella_europaea;yHMPu5000034606_Priceomyces_medius;yHMPu5000034607_Saccharomycopsis_capsularis;yHMPu5000034610_Saturnispora_hagleri;yHMPu5000034611_Saturnispora_mendoncae;yHMPu5000034612_Saturnispora_saitoi;yHMPu5000034613_Saturnispora_serradocipensis;yHMPu5000034614_Saturnispora_silvae;yHMPu5000034615_Saturnispora_zaruensis;yHMPu5000034622_Pichia_occidentalis;yHMPu5000034623_Pichia_norvegensis;yHMPu5000034624_Pichia_nakasei;yHMPu5000034625_Pichia_kudriavzevii;yHMPu5000034627_Pichia_heedii;yHMPu5000034629_Pichia_exigua;yHMPu5000034631_Martiniozyma_abiesophila;yHMPu5000034632_Candida_athensensis;yHMPu5000034635_Nadsonia_fulvescens;yHMPu5000034636_Ogataea_nitratoaversa;yHMPu5000034637_Ogataea_populiabae;yHMPu5000034643_Candida_schatavii;yHMPu5000034646_Wickerhamiella_cacticola;yHMPu5000034648_Candida_restingae;yHMPu5000034654_Aciculoconidium_aculeatum;yHMPu5000034655_Botryozyma_nematodophila;yHMPu5000034660_Diddensiella_caesifluorescens;yHMPu5000034661_Dipodascus_albidus;yHMPu5000034665_Kodamaea_laetipori;yHMPu5000034667_Blastobotrys_serpentis;yHMPu5000034669_Blastobotrys_raffinofermentans;yHMPu5000034670_Blastobotrys_proliferans;yHMPu5000034671_Blastobotrys_peoriensis;yHMPu5000034673_Blastobotrys_nivea;yHMPu5000034674_Blastobotrys_muscicola;yHMPu5000034675_Blastobotrys_mokoenaii;yHMPu5000034681_Blastobotrys_americana;yHMPu5000034742_Lipomyces_suomiensis;yHMPu5000034748_Lipomyces_oligophaga;yHMPu5000034749_Lipomyces_mesembrius;yHMPu5000034754_Lipomyces_arxii;yHMPu5000034760_Lipomyces_kononenkoae;yHMPu5000034761_Lipomyces_lipofer;yHMPu5000034883_Peterozyma_xylosa;yHMPu5000034884_Peterozyma_toletana;yHMPu5000034885_Ogataea_zsoltii;yHMPu5000034886_Ogataea_trehalophila;yHMPu5000034887_Ogataea_trehaloabstinens;yHMPu5000034890_Ogataea_ramenticola;yHMPu5000034891_Ogataea_pini;yHMPu5000034892_Ogataea_pilisensis;yHMPu5000034893_Ogataea_philodendra;yHMPu5000034897_Ogataea_glucozyma;yHMPu5000034899_Ogataea_kodamae;yHMPu5000034901_Ogataea_methylivora;yHMPu5000034902_Ogataea_minuta;yHMPu5000034903_Ogataea_naganishii;yHMPu5000034904_Ogataea_nonfermentans;yHMPu5000034918_Nakazawaea_holstii;yHMPu5000034933_Kuraishia_molischiana;yHMPu5000034939_Komagataella_pseudopastoris;yHMPu5000034946_Ambrosiozyma_oregonensis;yHMPu5000034947_Ambrosiozyma_philentoma;yHMPu5000034950_Citeromyces_hawaiiensis;yHMPu5000034952_Citeromyces_siamensis;yHMPu5000034957_Hanseniaspora_osmophila;yHMPu5000034963_Hanseniaspora_clermontiae;yHMPu5000034967_Candida_freyschussii;yHMPu5000034973_Danielozyma_ontarioensis;yHMPu5000034974_Deakozyma_indianensis;yHMPu5000034978_Cyberlindnera_mrakii;yHMPu5000034979_Cyberlindnera_misumaiensis;yHMPu5000034986_Candida_oregonensis;yHMPu5000034988_Candida_fructus;yHMPu5000034990_Candida_corydali;yHMPu5000034998_Cephaloascus_albidus;yHMPu5000034999_Cephaloascus_fragrans;yHMPu5000035011_Candida_pyralidae;yHMPu5000035018_Candida_canberraensis;yHMPu5000035022_Candida_emberorum;yHMPu5000035031_Candida_kruisii;yHMPu5000035032_Candida_gatunensis;yHMPu5000035033_Candida_cretensis;yHMPu5000035037_Candida_montana;yHMPu5000035040_Ambrosiozyma_maleeae;yHMPu5000035041_Ambrosiozyma_pseudovanderkliftii;yHMPu5000035044_Barnettozyma_californica;yHMPu5000035045_Barnettozyma_hawaiiensis;yHMPu5000035046_Barnettozyma_populi;yHMPu5000035047_Barnettozyma_pratensis;yHMPu5000035048_Barnettozyma_salicaria;yHMPu5000035242_Zygoascus_ofunaensis;yHMPu5000035243_Zygoascus_meyerae;yHMPu5000035244_Candida_incommunis;yHMPu5000035252_Yamadazyma_nakazawae;yHMPu5000035261_Candida_ponderosae;yHMPu5000035268_Wickerhamomyces_hampshirensis;yHMPu5000035271_Wickerhamomyces_bovis;yHMPu5000035274_Wickerhamomyces_alni;yHMPu5000035279_Tortispora_starmeri;yHMPu5000035282_Trigonopsis_vinaria;yHMPu5000035286_Candida_azyma;yHMPu5000035296_Priceomyces_carsonii;yHMPu5000035297_Priceomyces_castillae;yHMPu5000035301_Pichia_terricola;yHMPu5000035302_Candida_fragi;yHMPu5000035318_Hyphopichia_heimii;yHMPu5000035325_Cyberlindnera_petersonii;yHMPu5000035335_Candida_blattae;yHMPu5000035629_Yueomyces_sinensis;yHMPu5000035633_Candida_hispaniensis;yHMPu5000035639_Wickerhamomyces_canadensis;yHMPu5000035640_Yamadazyma_philogaea;yHMPu5000035641_Yamadazyma_scolyti;yHMPu5000035643_Yarrowia_bubula;yHMPu5000035645_Yarrowia_divulgata;yHMPu5000035650_Trigonopsis_variabilis;yHMPu5000035654_Tortispora_ganteri;yHMPu5000035658_Starmera_amethionina;yHMPu5000035659_Saturnispora_dispora;yHMPu5000035662_Meyerozyma_caribbica;yHMPu5000035665_Middelhovenomyces_tepae;yHMPu5000035667_Kurtzmaniella_cleridarum;yHMPu5000035670_Phaffomyces_opuntiae;yHMPu5000035671_Phaffomyces_antillensis;yHMPu5000035672_Phaffomyces_thermotolerans;yHMPu5000035673_Candida_orba;yHMPu5000035674_Kregervanrija_delftensis;yHMPu5000035675_Kregervanrija_fluxuum;yHMPu5000035677_Kodamaea_ohmeri;yHMPu5000035679_Candida_rhagii;yHMPu5000035681_Candida_gotoi;yHMPu5000035684_Kloeckera_hatyaiensis;yHMPu5000035686_Cyberlindnera_saturnus;yHMPu5000035687_Cyberlindnera_suaveolens;yHMPu5000035688_Cyberlindnera_xylosilytica;yHMPu5000035689_Candida_mycetangii;yHMPu5000035690_Candida_vartiovaarae;yHMPu5000035691_Candida_salmanticensis;yHMPu5000035695_Hanseniaspora_pseudoguilliermondii;yHMPu5000035696_Hanseniaspora_singularis;yHMPu5000035699_Cyberlindnera_maclurae;yHMPu5000035703_Cyberlindnera_americana;yHMPu5000035707_Candida_heveicola;yHMPu5000041678_Debaryomyces_prosopidis;yHMPu5000041693_Debaryomyces_nepalensis;yHMPu5000041713_Debaryomyces_maramus;yHMPu5000041743_Candida_hawaiiana;yHMPu5000041818_Magnusiomyces_tetrasperma;yHMPu5000041822_Dipodascus_geniculatus;yHMPu5000041824_Debaryomyces_subglobosus;yHMPu5000041829_Debaryomyces_fabryi;yHMPu5000041833_Candida_tammaniensis;yHMPu5000041840_Candida_wancherniae;yHMPu5000041855_Candida_ascalaphidarum;yHMPu5000041862_Candida_golubevii;yHMPu5000041863_Candida_gorgasii']
    species_onlyura9 = str(species_onlyura9).split(';')
    anaerobic = ['Sugiyamaella_lignohabitans', 'Dekkera_bruxellensis', 'yHMPu5000034625_Pichia_kudriavzevii',
                 'yHMPu5000026142_Citeromyces_matritensis', 'Candida_albicans', 'Candida_parapsilosis',
                 'Candida_tropicalis',
                 'Clavispora_lusitaniae', 'Spathaspora_passalidarum', 'Wickerhamia_fluorescens',
                 'Wickerhamomyces_anomalus', 'yHMPu5000035686_Cyberlindnera_saturnus', 'Hanseniaspora_uvarum',
                 'Hanseniaspora_valbyensis',
                 'Hanseniaspora_vinae', 'yHMPu5000034957_Hanseniaspora_osmophila', 'Ashbya_aceri', 'Candida_glabrata',
                 'Eremothecium_coryli', 'Kluyveromyces_lactis', 'Kluyveromyces_marxianus', 'Lachancea_fermentati',
                 'Lachancea_kluyveri', 'Lachancea_thermotolerans', 'Lachancea_waltii', 'Nakaseomyces_bacillisporus',
                 'Nakaseomyces_castellii', 'Nakaseomyces_delphensis', 'Naumovozyma_castellii',
                 'Naumovozyma_dairenensis',
                 'Saccharomyces_cerevisiae', 'Saccharomyces_eubayanus', 'Saccharomyces_paradoxus',
                 'Saccharomyces_uvarum', 'Tetrapisispora_blattae', 'Tetrapisispora_phaffii', 'Torulaspora_delbrueckii',
                 'Vanderwaltozyma_polyspora',
                 'Zygosaccharomyces_bailii', 'yHAB154_Kazachstania_transvaalensis',
                 'yHMPu5000034881_Torulaspora_pretoriensis', 'yHMPu5000034876_Tetrapisispora_iriomotensis',
                 'yHMPu5000034862_Zygotorulaspora_florentina',
                 'yHMPu5000026152_Torulaspora_franciscae', 'Schizosaccharomyces_pombe']
    species_intersect = set(anaerobic) & set(species_onlyura9)

    if len(set(species_intersect) & set(strain)):
        for i in range(len(model['rxns'])):
            if model['rxns'][i]=='r_2090':
                model['lb'][i] = -1000

    # 4rd change: Blocked pathways for proper glycerol production
    for s in range(len(model['rxns'])):
        # Block oxaloacetate-malate shuttle (not present in anaerobic conditions)
        if str(model['rxns'][s]).startswith('r_0713'):
            model['lb'][s] = 0  # Mithocondria
        if model['rxns'][s] == 'r_0714':
            model['lb'][s] = 0  # Cytoplasm
        if model['rxns'][s] == 'r_0713_rvs':
            model['lb'][s] = 0  # Mithocondria
        if model['rxns'][s] == 'r_0714_rvs':
            model['lb'][s] = 0  # Cytoplasm

        # Block glycerol dehydroginase (only acts in microaerobic conditions)
        if model['rxns'][s] == 'r_0487':
            model['ub'][s] = 0
        if model['rxns'][s] == 'r_0487_rvs':
            model['ub'][s] = 0

        # Block 2-oxoglutarate + L-glutamine -> 2 L-glutamate (alternative pathway)
        if model['rxns'][s] == 'r_0472':
            model['ub'][s] = 0
        if model['rxns'][s] == 'r_0472_fwd':
            model['ub'][s] = 0

    # 4th change: Blocked pathways for proper glycerol production
    # Block oxaloacetate-malate shuttle (not present in anaerobic conditions)
    for s in range(len(model['rxns'])):
        idx = []
        for t in range(len(model['rxns'])):
            if str(model['rxns'][t]).find('r_0713') >= 0 and str(model['rxns'][t]).find('rvs') >= 0:
                idx.append(t)
        if s in model['rxns'][idx]:
            model['ub'][s] = 0
            model['lb'][s] = 0
        if model['rxns'][s] == 'r_0713':
            model['lb'][s] = 0

    for s in range(len(model['rxns'])):
        idx = []
        for t in range(len(model['rxns'])):
            if str(model['rxns'][t]).find('r_0714') >= 0 and str(model['rxns'][t]).find('rvs') >= 0:
                idx.append(t)
        if s in model['rxns'][idx]:
            model['ub'][s] = 0
            model['lb'][s] = 0
        if model['rxns'][s] == 'r_0714':
            model['lb'][s] = 0

            # %Block 2-oxoglutarate + L-glutamine -> 2 L-glutamate (alternative pathway)

    for s in range(len(model['rxns'])):
        idx = []
        if str(model['rxns'][s]).startswith('r_0472_'):
            idx.append(s)
        if s in model['rxns'][idx]:
            model['ub'][s] = 0
            model['lb'][s] = 0
        if model['rxns'][s] == 'r_0472':
            model['ub'][s] = 0
    return model

def changeGAM(model, GAM, NGAM):
    bioPos = []
    for i in range(len(rxnNames)):
        if rxnNames[i]=='biomass pseudoreaction':
            bioPos.append(i)

    for i in range(len(model['mets'])):
        S_ix = model['S'][i, bioPos]
        isGAM_id = ['ATP [cytoplasm]', 'ADP [cytoplasm]', 'H2O [cytoplasm]', 'H+ [cytoplasm]', 'phosphate [cytoplasm]']
        if str(model['metNames'][i]) in isGAM_id:
            isGAM = True
        else:
            isGAM = False

        if S_ix!=0 and isGAM:
            if S_ix > 0:
                model['S'][i, bioPos] = GAM
            else:
                model['S'][i, bioPos] = -GAM

    for i in range(len(rxnNames)):
        if rxnNames[i]=='non-growth associated maintenance reaction':
            model_tmp["lb"][i] = NGAM
            model_tmp["ub"][i] = NGAM
    return model
